In [33]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

builder = SparkSession.builder \
    .appName("Delta Spark 4.0.0") \
    .master("local[*]") \
    .enableHiveSupport() \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [34]:
#CARGANDO INFORMACIÓN EN DATAFRAMES
dim_ventas = spark.read.format("delta").load("../sql/dw_silver/dim_ventas")
dim_productos = spark.read.format("delta").load("../sql/dw_silver/dim_productos")
dim_clientes = spark.read.format("delta").load("../sql/dw_silver/dim_clientes")
dim_territorio = spark.read.format("delta").load("../sql/dw_silver/dim_territorio")
dim_fecha = spark.read.format("delta").load("../sql/dw_silver/dim_fecha")

In [ ]:
#CREACIÓN DE TABLA DE HECHOS
from pyspark.sql.functions import date_format, col
df_hechos = dim_ventas.select(
    "ProductoID",
    "PedidoID",
    date_format(col("FechaPedido"), "yyyyMMdd").cast("int").alias("FechaID"),
    "ClienteID",
    "TerritorioID",
    "Cantidad_pedido",
    "Linea_total"
)
df_hechos.show()

In [37]:
df_hechos.write.option("mergeSchema", "true").format("delta").mode("overwrite").save("../sql/dw_gold/df_hechos")

In [ ]:
#OPERACIONES DE CONSULTAS
from pyspark.sql.functions import sum as _sum, round

#SUMA DE VENTAS POR CONTINENTE
df_ventas_continente = df_hechos.join(
    dim_territorio,
    on = "TerritorioID",
    how = "inner"
).groupBy("Continente").agg(
    round(_sum("Linea_total"), 2).alias("Suma_Total")
)
df_ventas_continente.show()

#CANTIDAD VENDIDA POR PRODUCTO - AÑADIR TIPO O CLASIFICACIÓN DE PRODUCTO
df_cant_vendida = df_hechos.join(
    dim_productos,
    on = "ProductoID",
    how = "Inner"
).groupBy("Nombre_producto").agg(
    round(_sum("Precio_lista"), 2).alias("Suma_Total")
)
df_cant_vendida.show()

#TOP 10 PRODUCTOS MÁS VENDIDOS - AÑADIR TIPO O CLASIFICACIÓN DE PRODUCTO
df_top = df_hechos.join(
    dim_productos,
    on = "ProductoID",
    how = "inner"
).groupBy("Nombre_producto").agg(
    round(_sum("Cantidad_pedido"),2).alias("Suma_Cantidad")
).orderBy("Suma_Cantidad", ascending = False).limit(10)
df_top.show()

#VENTAS AGRUPADAS POR CONTINENTE
df_ventas_agrupadas = df_hechos.join(
    dim_territorio,
    on = "TerritorioID",
    how = "inner"
).join(
    dim_productos,
    on = "ProductoID",
    how = "Inner"
).groupBy("Continente","Nombre_producto").agg(round(_sum("Linea_total"),2).alias("Costo_Ventas_Continente"))
df_ventas_agrupadas.show()

#VENTAS REALIZADAS POR CONTINENTES POR FECHA
df_ventas_fecha = df_hechos.join(
    dim_fecha,
    on = "FechaID",
    how = "Inner"
).join(
    dim_territorio,
    on = "TerritorioID",
    how = "inner"
).groupBy("Continente","Fecha").agg(round(_sum("Linea_total"),2).alias("Ventas"))
df_ventas_fecha.show()